### Loading our libraries 

In [44]:
#Importing Regular Expressions module 
import re

# Importing numpy 
import numpy as np

# ....... Pandas 
import pandas as pd

# ........ Tensorflow 
import tensorflow as tf
# ......... Matplotlib 
import matplotlib.pyplot as plt

# ..........Pandas 
import pandas as pd

# Importing train test split from Sklearn 
from sklearn.model_selection import train_test_split

import os

# load the libraries
import numpy as np

import warnings
warnings.filterwarnings("ignore")
import cv2
from tqdm import tqdm

from PIL import Image

import sys

import csv


### Data Loading

In [45]:
DATADIR = "/Users/cyprian/Documents/covid19-pneumonia-normal-chest-xraypa-dataset 2/COVID19_Pneumonia_Normal_Chest_Xray_PA_Dataset"


CATEGORIES = ['covid', 'normal', 'pneumonia']






In [46]:
#Useful function
def createFileList(myDir, format='.jpg'):
    fileList = []
    print(myDir)

    for root, dirs, files in os.walk(myDir, topdown=False):

        for name in files:

            if name.endswith(format):

                fullName = os.path.join(root, name)
                fileList.append(fullName)
    return fileList




### Visualize the data 

In [47]:
for category in CATEGORIES: 
    path = os.path.join(DATADIR,category) 
    for img in os.listdir(path): 
        img_array = cv2.imread(os.path.join(path,img) ) 
        plt.imshow(img_array) 
        plt.show() 

        break 
    break  

In [48]:
# img_array

In [49]:
IMG_SIZE = 200

new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
plt.imshow(new_array)
plt.show()

### Building our training data!

In [50]:
training_data = []

def create_training_data():
    for category in CATEGORIES: 

        path = os.path.join(DATADIR,category)  
        class_num = CATEGORIES.index(category)  

        for img in tqdm(os.listdir(path)): 
            try:
                img_array = cv2.imread(os.path.join(path,img) ,cv2.IMREAD_GRAYSCALE) 
                new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))  
                training_data.append([new_array, class_num]) 
            except Exception as e: 
                pass
           

create_training_data()

print(len(training_data))

              

100%|██████████| 100/100 [00:02<00:00, 46.43it/s]

300


### Shuffling our data 

Next, we want to shuffle the data. Right now our data is just all pneumonia, then normal and covid. This will usually end up causing trouble as, initially, the classifier will learn to just predict  pneumonia, normal and covid

In [51]:
import random

random.shuffle(training_data)

### Viewing  the shaffled data.

In [52]:
for sample in training_data[:100]:
    
    print(sample[1])

0
0
1
0
2
1
0
2
2
1
0
0
1
1
0
0
0
0
2
2
0
1
2
2
1
2
2
1
0
1
1
1
1
2
0
0
0
0
2
2
1
0
2
0
1
1
2
2
1
0
0
1
0
2
1
2
1
2
1
0
0
0
1
1
1
2
1
1
1
2
0
0
2
0
1
1
0
1
0
1
2
2
1
0
0
0
1
2
0
2
0
2
2
1
1
0
1
0
0
2


### Making our model 

In [53]:
X=[]
y=[]

In [54]:

for features,label in training_data:
    X.append(features)
    y.append(label)

print(X[0].reshape(-1, IMG_SIZE, IMG_SIZE, 1))
X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
y = np.array(y)

[[[[19]
   [22]
   [24]
   ...
   [ 2]
   [ 0]
   [ 1]]

  [[10]
   [11]
   [12]
   ...
   [ 1]
   [ 0]
   [ 1]]

  [[15]
   [15]
   [16]
   ...
   [85]
   [77]
   [ 1]]

  ...

  [[ 0]
   [ 0]
   [ 0]
   ...
   [ 0]
   [ 0]
   [ 1]]

  [[ 0]
   [ 0]
   [ 0]
   ...
   [ 0]
   [ 0]
   [ 1]]

  [[ 0]
   [ 0]
   [ 0]
   ...
   [ 0]
   [ 0]
   [ 1]]]]


In [55]:
import pickle

pickle_out = open("X.pickle","wb")
pickle.dump(X, pickle_out)
pickle_out.close()

pickle_out = open("y.pickle","wb")
pickle.dump(y, pickle_out)
pickle_out.close()

### Loading our current scripts 

### Making the predictions 

In [56]:
# Loading the libraries 
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D

from tensorflow.keras.callbacks import TensorBoard

In [58]:



pickle_in = open("X.pickle","rb")
X = pickle.load(pickle_in)

pickle_in = open("y.pickle","rb")
y = pickle.load(pickle_in)

X = X/255.0

model = Sequential()

model.add(Conv2D(256, (3, 3), input_shape=X.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(256, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors

model.add(Dense(64))

model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.fit(X, y, batch_size=32, epochs=3, validation_split=0.3, callbacks=[TensorBoard])

AttributeError: 'str' object has no attribute 'set_model'

### Final Predictions 